In [7]:
from pyproj import Proj, transform, CRS, Transformer
import math

points = [50.016666666666666, 18.016666666666666, 51.35954501388889, 19.0, 51.350750175, 20.435489580555554, 50.991204616666664, 20.435489580555554]

e2 = 0.00669438002290
a = 6378137
b2 = a**2 * (1 - e2)
eprim2 = (a**2 - b2) / b2

A0 = 1 - e2/4 - 3*e2**2/64 - 5*e2**3/256
A2 = 3/8 * (e2 + e2**2/4 + 15*e2**3/128)
A4 = 15/256 * (e2**2 + 3*e2**3/4)
A6 = 35*e2**3/3072

def gk (fi, lam, lam0):
    # 2. wielkości pomocnicze
    delta_lam = lam - lam0
    t = math.tan(fi)
    eta2 = eprim2 * math.cos(fi)**2
    N = a / math.sqrt(1-e2*math.sin(fi)**2)

    sigma = a * (A0 * fi - A2 * math.sin(2 * fi) + A4 * math.sin(4 * fi) - A6 * math.sin(6 * fi))
    
    # 4. współrzędne prostokątne lokalne na płaszczyźnie Gaussa-Krügera
    xgk = sigma + (delta_lam**2 / 2 * N * math.sin(fi) * math.cos(fi)) * (1 + delta_lam**2 / 12 * math.cos(fi)**2 * (5 - t**2 + 9*eta2 + 4*eta2**2) + delta_lam**4 / 360 * math.cos(fi)**4 * (61 - 58*t**2 + t**4 + 270*eta2 - 330*t**2*eta2))

    ygk = delta_lam * N * math.cos(fi) * (1 + delta_lam**2 / 6 * math.cos(fi)**2 * (1 - t**2 + eta2) + delta_lam**4 / 120 * math.cos(fi)**4 * (5 - 18*t**2 + t**4 + 14*eta2 - 58*t**2*eta2 + 13*eta2**2 + 4*eta2**3))

    return xgk, ygk

def from_gk(xgk, ygk, lam0):
    fi = xgk / (a * A0)
    sigma = a * (A0 * fi - A2 * math.sin(2 * fi) + A4 * math.sin(4 * fi) - A6 * math.sin(6 * fi))

    while True:
        fi1 = fi + (xgk - sigma) / (a * A0)

        N = a / math.sqrt(1-e2*math.sin(fi1)**2)
        M = a * (1 - e2) / math.sqrt(1-e2*math.sin(fi1)**2)**3
        t = math.tan(fi1)
        eta2 = eprim2 * math.cos(fi1)**2
        sigma = a * (A0 * fi1 - A2 * math.sin(2 * fi1) + A4 * math.sin(4 * fi1) - A6 * math.sin(6 * fi1))

        if abs(fi1 - fi) < (0.000001 / 3600):
            break

        fi = fi1

    fi = fi1 - ((ygk**2 * t) / (2 * M * N)) * (1 - ygk**2 / 12 * N**2 * (5 + 3*t**2 + eta2 - 9*t**2*eta2 - 4*eta2**2) + ygk**4 / 360 * N**4 * (61 + 90*t**2 + 45*t**4))

    lam = lam0 + (ygk / (N * math.cos(fi))) * (1 - ygk**2 / 6 * N**2 * (1 + 2*t**2 + eta2) + ygk**4 / 120 * N**4 * (5 + 28*t**2 + 24*t**4 + 6*eta2 + 8*t**2*eta2))

    return fi, lam


    
